## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_map

# Configure gmaps API key
gmaps.configure(api_key=g_map)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lewiston,US,44.1003,-70.2148,58.80,76,90,3.44,overcast clouds
1,1,Kalmunai,LK,7.4167,81.8167,80.82,78,78,2.75,light rain
2,2,Bluff,NZ,-46.6000,168.3333,47.80,82,90,20.74,overcast clouds
3,3,Inhambane,MZ,-23.8650,35.3833,75.18,73,20,0.00,few clouds
4,4,Pangai,TO,-19.8000,-174.3500,77.11,69,75,13.80,broken clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lewiston,US,44.1003,-70.2148,58.80,76,90,3.44,overcast clouds
3,3,Inhambane,MZ,-23.8650,35.3833,75.18,73,20,0.00,few clouds
4,4,Pangai,TO,-19.8000,-174.3500,77.11,69,75,13.80,broken clouds
5,5,Avarua,CK,-21.2078,-159.7750,68.05,100,5,2.30,clear sky
6,6,Port Alfred,ZA,-33.5906,26.8910,65.68,90,100,19.42,light rain
11,11,Rio Grande,BR,-32.0350,-52.0986,68.11,77,89,24.02,overcast clouds
12,12,Vanimo,PG,-2.6741,141.3028,77.20,89,100,1.50,overcast clouds
14,14,Datong,CN,40.0936,113.2914,54.61,39,1,6.51,clear sky
16,16,Busselton,AU,-33.6500,115.3333,60.04,88,29,15.10,light rain
17,17,Noumea,NC,-22.2763,166.4572,73.99,99,47,8.86,scattered clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                378
City                   378
Country                373
Lat                    378
Lng                    378
Max Temp               378
Humidity               378
Cloudiness             378
Wind Speed             378
Current Description    378
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                373
City                   373
Country                373
Lat                    373
Lng                    373
Max Temp               373
Humidity               373
Cloudiness             373
Wind Speed             373
Current Description    373
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lewiston,US,58.80,overcast clouds,44.1003,-70.2148,
3,Inhambane,MZ,75.18,few clouds,-23.8650,35.3833,
4,Pangai,TO,77.11,broken clouds,-19.8000,-174.3500,
5,Avarua,CK,68.05,clear sky,-21.2078,-159.7750,
6,Port Alfred,ZA,65.68,light rain,-33.5906,26.8910,
11,Rio Grande,BR,68.11,overcast clouds,-32.0350,-52.0986,
12,Vanimo,PG,77.20,overcast clouds,-2.6741,141.3028,
14,Datong,CN,54.61,clear sky,40.0936,113.2914,
16,Busselton,AU,60.04,light rain,-33.6500,115.3333,
17,Noumea,NC,73.99,scattered clouds,-22.2763,166.4572,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_map
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel Not Found....skipping.")
        

Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lewiston,US,58.80,overcast clouds,44.1003,-70.2148,Hilton Garden Inn Auburn Riverwatch
3,Inhambane,MZ,75.18,few clouds,-23.8650,35.3833,Oceano
4,Pangai,TO,77.11,broken clouds,-19.8000,-174.3500,Ha'apai Beach Resort
5,Avarua,CK,68.05,clear sky,-21.2078,-159.7750,Paradise Inn
6,Port Alfred,ZA,65.68,light rain,-33.5906,26.8910,The Halyards Hotel


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))